<a href="https://www.kaggle.com/code/osmanacar/online-retail-rfm-analysis?scriptVersionId=187677919" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Business Problem**

A company wants to segment its customers based on their purchase history information. 

The aim is to provide recommendations to customers based on their most recent purchase dates.

**Variables**

* InvoiceNo: Unique invoice number. If includes C, invoice were cancelled
* StockCode: Unique product code
* Description: Name of product
* Quantity: Number of product 
* InvoiceDate: Invoice date that was purchased
* UnitPrice: Cost of product
* CustomerID: Unique customer number
* Country: Country name that customer lives in

In [1]:
import datetime as dt
import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
df_ = pd.read_excel("/kaggle/input/online-retail/online_retail_II.xlsx")
df = df_.copy()

In [3]:
# Data control
def check_df(dataframe, head=5):
    print("################# shape #################")
    print(dataframe.shape)
    print("################# types #################")
    print(dataframe.dtypes)
    print("################# head #################")
    print(dataframe.head(head))
    print("################# tail #################")
    print(dataframe.tail(head))
    print("################# na #################")
    print(dataframe.isnull().sum())
    print("################# quantiles #################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################# shape #################
(525461, 8)
################# types #################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
################# head #################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00 6.9500   13085.0000  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00 6.7500   13085.0000  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00 6.7500   13085.0000  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00 2.1000   13085.0000  United Ki

In [4]:
# For outliers
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range

    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [5]:
# Cleaning some unnecessary values from data

df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[~df["StockCode"].str.contains("POST", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

/tmp/ipykernel_33/3484166592.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-213.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit


In [6]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,406926.0000,11.9227,1.0000,2.0000,5.0000,12.0000,358.5000,28.7485
InvoiceDate,406926,2010-07-01 09:58:01.902655744,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:31:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,406926.0000,2.9644,0.0010,1.2500,1.9500,3.7500,31.4400,3.0573
Customer ID,406926.0000,15373.4599,12346.0000,14006.0000,15325.0000,16814.0000,18287.0000,1677.2695


In [7]:
df.shape

(406926, 8)

In [8]:
# Data preperation for RFM Analysis
df["TotalPrice"] = df["Quantity"] * df["Price"]

df["InvoiceDate"].max()

today_date = df["InvoiceDate"].max() + pd.DateOffset(2)

rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda InvoiceDate:(today_date - InvoiceDate.max()).days,
    "Invoice": lambda Invoice: Invoice.nunique(),
    "TotalPrice": lambda TotalPrice: TotalPrice.sum()
})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0000,166,11,372.8600
12347.0000,4,2,1323.3200
12348.0000,75,1,221.1600
12349.0000,44,2,2207.1000
12351.0000,12,1,300.9300


In [9]:
# We are gonna change columns names
rfm.columns = ["recency", "frequency", "monetary"]

In [10]:
# Creating RFM scores. We care just two information. These are recency and frequency. 
# Monetary is necessary for this case

rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[5, 4, 3, 2, 1])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[5, 4, 3, 2, 1])

rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.0000,166,11,372.8600,2,1,4,21
12347.0000,4,2,1323.3200,5,4,2,54
12348.0000,75,1,221.1600,2,5,5,25
12349.0000,44,2,2207.1000,3,4,1,34
12351.0000,12,1,300.9300,5,5,4,55


In [11]:
# Adding segment. 

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)

rfm.head(20)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0000,166,11,372.8600,2,1,4,21,hibernating
12347.0000,4,2,1323.3200,5,4,2,54,champions
12348.0000,75,1,221.1600,2,5,5,25,cant_loose
12349.0000,44,2,2207.1000,3,4,1,34,loyal_customers
12351.0000,12,1,300.9300,5,5,4,55,champions
12352.0000,12,2,343.8000,5,4,4,54,champions
12353.0000,45,1,317.7600,3,5,4,35,loyal_customers
12355.0000,204,1,488.2100,1,5,4,15,cant_loose
12356.0000,17,3,3126.2500,4,3,1,43,potential_loyalists


In [12]:
# Segment information.
rfm[["segment", "recency", "frequency", "monetary",]].groupby("segment").agg(["mean", "count"])

recency       frequency        monetary      
                        mean count      mean count      mean count
segment                                                           
about_to_sleep       54.4120   301    6.1096   301 2305.3726   301
at_Risk             189.5281   835    1.7509   835  569.0870   835
cant_loose          217.8227   547    1.0000   547  322.6283   547
champions            10.0248   121    1.2645   121  456.9510   121
hibernating         135.1807   321    5.3209   321 2010.5685   321
loyal_customers      42.8812   589    1.2462   589  466.3280   589
need_attention       54.2956   203    2.4384   203  992.9950   203
new_customers         7.6214   457   16.1751   457 8809.8003   457
potential_loyalists  18.2392   719    3.5897   719 1255.8791   719
promising            25.5899   217   10.3410   217 4244.2968   217